### Prelimniraries
1. Mounting Google Drive
2. Importing required libraries, I don't really know what I need so I'll add as I go.


In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')
!ln -s '/content/drive/My Drive/InterIIT' '/content/InterIIT'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ln: failed to create symbolic link '/content/InterIIT/InterIIT': File exists


Downloading Facebook's InferSent

In [ ]:
import nltk
import pandas as pd
from textblob import TextBlob
import torch
import numpy as np
import copy
import datetime
import os
import sys
import pickle
from collections import namedtuple
import datetime
import numpy as np
!pip install annoy
import tensorflow as tf
import tensorflow_hub as hub
import annoy
from sklearn.random_projection import GaussianRandomProjection
import torch
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
datetime.datetime.now()

datetime.datetime(2022, 12, 25, 9, 36, 36, 30013)

In [ ]:
# !mkdir GloVe
# !curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
# !unzip GloVe/glove.840B.300d.zip -d GloVe/
# !mkdir fastText
# !curl -Lo fastText/crawl-300d-2M.vec.zip https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
# !unzip fastText/crawl-300d-2M.vec.zip -d fastText/

In [ ]:
# !mkdir encoder
# !curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
# !curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

In [ ]:
!wget https://raw.githubusercontent.com/facebookresearch/InferSent/main/models.py

--2022-12-25 10:10:58--  https://raw.githubusercontent.com/facebookresearch/InferSent/main/models.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9875 (9.6K) [text/plain]
Saving to: ‘models.py.3’

models.py.3         100%[===================>]   9.64K  --.-KB/s    in 0s      

2022-12-25 10:10:58 (65.0 MB/s) - ‘models.py.3’ saved [9875/9875]



In [ ]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [ ]:
# !unzip '/content/drive/MyDrive/InterIIT/fastText/crawl-300d-2M.vec.zip'

In [ ]:
# !cp -r '/content/crawl-300d-2M.vec' '/content/drive/MyDrive/InterIIT/fastText/crawl-300d-2M.vec'

InferSent Model Setup

In [ ]:
from models import InferSent
V = 2
MODEL_PATH = '/content/InterIIT/encoder/infersent2.pkl'
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [ ]:
W2V_PATH = '/content/InterIIT/fastText/crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)

In [ ]:
infersent = infersent.cuda()

### Basic EDA

In [ ]:
basic_dataframe = pd.read_csv('/content/InterIIT/train_data.csv')

In [ ]:
def break_into_sentences(text):
  blob = TextBlob(text)
  sent = [str(i) for i in blob.sentences]
  return sent

In [ ]:
basic_dataframe.columns

Index(['Unnamed: 0', 'Theme', 'Paragraph', 'Question', 'Answer_possible',
       'Answer_text', 'Answer_start'],
      dtype='object')

In [ ]:
# basic_dataframe["tokenized"] = basic_dataframe.apply(lambda x: break_into_sentences(x['Paragraph']), axis = 1)

In [ ]:
unique_para = basic_dataframe["Paragraph"].unique()

In [ ]:
len(unique_para)

15555

In [ ]:
t1 = datetime.datetime.now()
sentences = []
questions = []
for i in range(len(unique_para)):
  sent = break_into_sentences(unique_para[i])
  if i%5000 == 0:
    print(i, "done")
    # print(sent)
  sentences += sent
for i in range(len(basic_dataframe)):
  ques = str(basic_dataframe["Question"].iloc[i])
  # if i%50 == 0:
    # print(ques)
  questions += [ques]
t2 = datetime.datetime.now()
print(t2 - t1)

0 done
5000 done
10000 done
15000 done
0:00:08.974595


In [ ]:
len(sentences)

77838

In [ ]:
t1 = datetime.datetime.now()
infersent.build_vocab(sentences + questions, tokenize = True)
t2 = datetime.datetime.now()
print(t2 - t1)

Found 84735(/103964) words with w2v vectors
Vocab size : 84735
0:00:46.114807


In [ ]:
sent_embeddings = []

In [ ]:
len(sentences)/128

608.109375

In [ ]:
for i in range(0, len(sentences), 128):
  sent_embeddings += [infersent.encode(sentences[i:i+128], bsize = 128 ,tokenize = True, verbose = True)]

Nb words kept : 3757/3894 (96.5%)


/content/models.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentences = np.array(sentences)[idx_sort]


Speed : 141.6 sentences/s (gpu mode, bsize=128)
Nb words kept : 3653/3793 (96.3%)
Speed : 722.0 sentences/s (gpu mode, bsize=128)
Nb words kept : 3996/4140 (96.5%)
Speed : 721.2 sentences/s (gpu mode, bsize=128)
Nb words kept : 3545/3645 (97.3%)
Speed : 878.2 sentences/s (gpu mode, bsize=128)
Nb words kept : 3332/3408 (97.8%)
Speed : 807.3 sentences/s (gpu mode, bsize=128)
Nb words kept : 2942/3008 (97.8%)
Speed : 989.1 sentences/s (gpu mode, bsize=128)
Nb words kept : 4511/4711 (95.8%)
Speed : 662.6 sentences/s (gpu mode, bsize=128)
Nb words kept : 4555/4780 (95.3%)
Speed : 675.7 sentences/s (gpu mode, bsize=128)
Nb words kept : 4129/4290 (96.2%)
Speed : 726.0 sentences/s (gpu mode, bsize=128)
Nb words kept : 3449/3504 (98.4%)
Speed : 974.4 sentences/s (gpu mode, bsize=128)
Nb words kept : 3702/3736 (99.1%)
Speed : 854.4 sentences/s (gpu mode, bsize=128)
Nb words kept : 4081/4164 (98.0%)
Speed : 690.2 sentences/s (gpu mode, bsize=128)
Nb words kept : 3904/3930 (99.3%)
Speed : 674.6 se

In [ ]:
len(sent_embeddings)

In [ ]:
# quest_embeddings = []
# for i in range(0, len(questions), 128):
#   quest_embeddings.append(infersent.encode(questions[i:i+128], bsize = 128 ,tokenize = True, verbose = True))

In [ ]:
q = infersent.encode(questions, bsize = 128 ,tokenize = True, verbose = True)

Nb words kept : 974825/982207 (99.2%)
Speed : 1880.7 sentences/s (gpu mode, bsize=128)


In [ ]:
len(q)

In [ ]:
sent = []
for i in sent_embeddings:
  for j in i:
    sent.append(j)

77838

In [ ]:
from numpy import dot
from numpy.linalg import norm
def cos_sim(a,b):
  return dot(a, b)/(norm(a)*norm(b))


In [ ]:
%%timeit
cos_sim(q[1], sent[1])

11.4 µs ± 347 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
def get_best_match(q_index):
  base_cos = -2
  ind = -6
  for i in range(len(sent)):
    cos = cos_sim(q[q_index], sent[i])
    if cos > base_cos:
      ind = i
      base_cos = cos
  return(sentences[ind], questions[q_index])


In [ ]:
build_index("beyoncebbb", ratio = 5)

5000 items loaded
10000 items loaded
15000 items loaded
Loading completed suckers
Building??????
Index file size: 0.3 GB


In [ ]:
index = annoy.AnnoyIndex(4096)
index.load('beyoncebbb', prefault=True)
print('Annoy index is loaded.')


Annoy index is loaded.


<ipython-input-27-150ece8400de>:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  index = annoy.AnnoyIndex(4096)


In [ ]:
def find_similar_items(embedding, num_matches=25):
  '''Finds similar items to a given embedding in the ANN index'''
  ids = index.get_nns_by_vector(
  embedding, num_matches, search_k=-1, include_distances=False)
  return ids

In [ ]:
# %%timeit
x = find_similar_items(q[0])
y = [sentences[i] for i in x]

In [ ]:
for i in y:
  print(i)

print("\n\n")
print(questions[0])

Beyoncé's younger sister Solange is also a singer and a former member of Destiny's Child.
When Beyoncé does an album, when Beyoncé sings a song, when Beyoncé does anything, it's an event, and it's broadly influential.
When The Guardian named her Artist of the Decade, Llewyn-Smith wrote, "Why Beyoncé?
The album's lead single, "Crazy in Love", featuring Jay Z, became Beyoncé's first number-one single as a solo artist in the US.
Beyoncé's third solo album I Am... Sasha Fierce was inspired by Jay Z and especially by Etta James, whose "boldness" inspired Beyoncé to explore other musical genres and styles.
Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress.
The feminism and female empowerment themes on Beyoncé's second solo album B'Day were inspired by her role in Dreamgirls and by singer Josephine Baker.
To promote the film, Beyoncé released "Listen" as the lead single from the soundtrack album.
B

In [ ]:
s = np.array(sent)
# q = np.array(q)

In [ ]:
s.shape

(77838, 4096)

In [ ]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import faiss

In [ ]:
faiss.normalize_L2(s)

In [ ]:
index = faiss.index_factory(4096, "Flat", faiss.METRIC_INNER_PRODUCT)

In [ ]:
index.add(s)

In [ ]:
q_ = np.array(q)

In [ ]:
faiss.normalize_L2(q_)

In [ ]:
ll = np.array([list(q_[0])])

In [ ]:
D, I = index.search(ll, 12)  # search
print(I)


[[   19   296   292    75   246 35980 35811     0   244    93    53  3938]]


In [ ]:
for i in I:
  for j in I:
    for x_ in range(len(j)):
      print(sentences[j[x_]])

Beyoncé's younger sister Solange is also a singer and a former member of Destiny's Child.
When Beyoncé does an album, when Beyoncé sings a song, when Beyoncé does anything, it's an event, and it's broadly influential.
When The Guardian named her Artist of the Decade, Llewyn-Smith wrote, "Why Beyoncé?
The album's lead single, "Crazy in Love", featuring Jay Z, became Beyoncé's first number-one single as a solo artist in the US.
Beyoncé's third solo album I Am... Sasha Fierce was inspired by Jay Z and especially by Etta James, whose "boldness" inspired Beyoncé to explore other musical genres and styles.
When The Beatles hit America, they changed the paradigm of performer from solo act to band.
Days later, she joined singer Miley Cyrus on her MTV Unplugged special, singing a mash-up of "Don't Tell Me" and Cyrus' single "We Can't Stop" (2013).
Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress.
Th

In [ ]:
questions[0]

"When did Beyonce leave Destiny's Child and become a solo singer?"

In [ ]:
# def best_match_cosine(embeddings):
#   question_embedding = embeddings[-1]
#   mag_q = np.sqrt(np.sum(question_embedding**2))
#   max_cosine = 0
#   best_sent = -9
#   second_best = 0
#   second_sent = -9
#   for i in embeddings[:-1]:
#     dot_prod = np.dot(i, question_embedding)
#     mag_i = np.sqrt(np.sum(i**2))
#     cos_sim = dot_prod/(mag_i*mag_q)
#     # print(cos_sim)
#     if (cos_sim > max_cosine):
#       second_best = max_cosine
#       second_sent = best_sent
#       max_cosine = cos_sim
#       best_sent = i
      
#     # print(cos_sim)
#     # print(max_cosine)
#     # print(best_sent)
#     # print()
#   # print(best_sent)
#   # print(second_sent)
#   best_int = -9
#   second_int = None
#   for i in range(len(embeddings)):
#   # print(encodings[i])
#   # print(best_sent)
#     if (embeddings[i] == best_sent).all() == True:
#       best_int = i
#     # if (second_sent != -9):
#     #   if (embeddings[i] == second_sent).all() == True:
#     #     second_int = i
#   return best_int


  


In [ ]:
# def evaluate(index : int):
#   # print(index)
#   sentences, question, answer = get_ith_datapt(index)
#   # if possible:
#   embeddings = get_embeddings(sentences, question)
#   best_match = best_match_cosine(embeddings)
#   # output = "I: {}\nQ: {}\nA: {}\nBest Match: {}\n".format(index, question, answer, sentences[best_match])
#   output = sentences[best_match]
#   # if best_match[1] != None:
#   #   print("S2: ", sentences[best_match[1]])

#   # print("Ch: ", possible)
#   return output
#   # else:
#   #   output = "Answer not Possible lmaooo\n"
#   #   print("Answer not Possible lmaoooo")
#   #   return output
#   # for i in sentences:
#   #   print(i)


In [ ]:
import random

In [ ]:
import json